In [ ]:
#Here we build a pandas dataframe ufc_fighters containing all current fighter stats
#To run this, first we should have a fully updated fighter_stats.csv and ufcfightscrap

In [21]:
import pandas as pd
import numpy as npy
from datetime import date, datetime
#the input is the csv files fight_hist and fighter_stats which we scrape from ufcstats.com
#careful, might say file does not exist if its saved to cloud (not on local device)
ufcfightscrap=pd.read_csv('ufcfightscrap.csv',sep=',')
ufcfighterscrap=pd.read_csv('fighter_stats.csv',sep=',')
mask=ufcfighterscrap['dob']=='--'
ufcfighters=ufcfighterscrap[mask]

In [29]:
#this cell contains all functions defined for building columns in ufcfightscrap
#converts from '%B %d, %Y' (i.e. August 22, 2020) to date (i.e. 2020-08-22)
def convert_to_datetime(day1):
    return datetime.strptime(day1, '%B %d, %Y').date()
convert_to_datetime_vect= npy.vectorize(convert_to_datetime)

def age(birthDate,day=date.today(),form1='%b %d, %Y',form2='%B %d, %Y'):
    if birthDate=='--':
        aa='--'
    elif type(birthDate)==str and not type(day)==str:
        bd=datetime.strptime(birthDate, form1)
        today = day
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    elif type(birthDate)==str and type(day)==str:
        bd=datetime.strptime(birthDate, form1)
        today = datetime.strptime(day, form2)
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    elif not type(birthDate)==str and type(day)==str:
        bd=birthDate
        today = datetime.strptime(day, form2)
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    else:
        bd=birthDate
        today = day
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    return aa

age_vect= npy.vectorize(age)

def wins_before(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
    return summ

def losses_before(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
    return summ

#functions for height and reach
def fighter_height(fighter):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['height'][i]
            break
    if a=='--':
        b='unknown'
    elif a[4]=='"':
        b=int(a[0])*30.48+int(a[3])*2.54
    else:
        b=int(a[0])*30.48+int(a[3]+a[4])*2.54
    return b

def fighter_reach(fighter):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['reach'][i]
            break
    if a=='--':
        b='unknown'
    else:
        b=int(a[0]+a[1])*2.54
    return b

wins_before_vect= npy.vectorize(wins_before)
losses_before_vect= npy.vectorize(losses_before)
fighter_height_vect= npy.vectorize(fighter_height)
fighter_reach_vect= npy.vectorize(fighter_reach)

#day1 should be input in the form '%B %d, %Y' i.e. 'August 20, 1962'
#conversions can be made via day=datetime.strptime(ufcfightsML_known_df['date'][i], '%B %d, %Y').strftime('%b %d, %Y')
def time_diff(day1,day2=date.today()):
    if day2==date.today():
        answer=(day2-datetime.strptime(day, '%B %d, %Y')).days
    else:
        answer=(datetime.strptime(day2, '%B %d, %Y')-datetime.strptime(day1, '%B %d, %Y')).days
    return answer

#we now vectorize this to use in pandas/numpy
time_diff_vect= npy.vectorize(time_diff)

#can make a single function to do all of these... actually maybe the count function would even work as is

def L5Y_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L5Y_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L2Y_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

def L2Y_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

def ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
    return summ

def ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
    return summ

def L5Y_ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L5Y_ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L2Y_ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

def L2Y_ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

def sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
    return summ

def sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
    return summ

def L5Y_sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L5Y_sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L2Y_sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

def L2Y_sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

#vectorize all of these functions
L5Y_wins_vect= npy.vectorize(L5Y_wins)
L5Y_losses_vect= npy.vectorize(L5Y_losses)
L2Y_wins_vect= npy.vectorize(L2Y_wins)
L2Y_losses_vect= npy.vectorize(L2Y_losses)
ko_wins_vect= npy.vectorize(ko_wins)
ko_losses_vect= npy.vectorize(ko_losses)
L5Y_ko_wins_vect= npy.vectorize(L5Y_ko_wins)
L5Y_ko_losses_vect= npy.vectorize(L5Y_ko_losses)
L2Y_ko_wins_vect= npy.vectorize(L2Y_ko_wins)
L2Y_ko_losses_vect= npy.vectorize(L2Y_ko_losses)
sub_wins_vect= npy.vectorize(sub_wins)
sub_losses_vect= npy.vectorize(sub_losses)
L5Y_sub_wins_vect= npy.vectorize(L5Y_sub_wins)
L5Y_sub_losses_vect= npy.vectorize(L5Y_sub_losses)
L2Y_sub_wins_vect= npy.vectorize(L2Y_sub_wins)
L2Y_sub_losses_vect= npy.vectorize(L2Y_sub_losses)

#for columns like fighter_rec which contains the information for the opponent as well, we use the following
def opponent_column(stat):
    col=dict()
    for i in range(len(ufcfightscrap['fighter'])):
        if i%2==0:
            col[i]=ufcfightscrap[stat][i+1]
        else:
            col[i]=ufcfightscrap[stat][i-1]
    statdict={'stat':col}
    return pd.DataFrame (statdict, columns = ['stat'])

#enter date unabbreviated 'July 4, 2019'
#here the average gives avg per fight. Later in avg_count we change to average per time spent in octagon
def count(stat, guy,inf_abs, total_L5Y_L2Y_avg, day1=date.today().strftime('%B %d, %Y')):
    summ=0
    if total_L5Y_L2Y_avg=='total' or total_L5Y_L2Y_avg=='avg':
        good_indices_1=[i for i in ufcfightscrap.index.values if time_diff(ufcfightscrap['date'][i],day1)>0]
    elif total_L5Y_L2Y_avg=='L2Y':
        good_indices_1=[i for i in ufcfightscrap.index.values if 0<time_diff(ufcfightscrap['date'][i],day1)<730]
    else:
        good_indices_1=[i for i in ufcfightscrap.index.values if 0<time_diff(ufcfightscrap['date'][i],day1)<1825]
    if inf_abs=='inf':
        good_indices_2=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    else:
        good_indices_2=[i for i in ufcfightscrap.index.values if ufcfightscrap['opponent'][i]==guy]
    good_indices=[i for i in good_indices_1 if i in good_indices_2]
    if total_L5Y_L2Y_avg!='avg':
        for i in good_indices:
            summ+=ufcfightscrap[stat][i]
    else:
        for i in good_indices:
            summ+=ufcfightscrap[stat][i]
        day1=convert_date_to_abbrev(day1)
        number_fights=wins_before(guy,day1)+losses_before(guy,day1)
        summ=summ/float(number_fights)
    return summ
        
#note a better average is per time not per fight. We will do each stat as an average per one minutes

#enter time in the form 'August 24, 2018'
def time_in_octagon(guy,day1=date.today().strftime('%B %d, %Y')):
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['fighter'][i]==guy] 
    for i in good_indices:
        if ufcfightscrap['time'][i][2]==':':
            summ=int(ufcfightscrap['time'][i][0:2])+int(ufcfightscrap['time'][i][3:])/60.0
        else:
            summ+=5*(ufcfightscrap['round'][i]-1)+int(ufcfightscrap['time'][i][0])+int(ufcfightscrap['time'][i][2:])/60.0
    return summ

#enter date unabbreviated 'July 4, 2019'
#gives takedowns per minute
def avg_count(stat, guy,inf_abs, day1=date.today().strftime('%B %d, %Y')):
    summ=0
    if inf_abs=='inf':
        good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy and time_diff(ufcfightscrap['date'][i],day1)>0]
    else:
        good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['opponent'][i]==guy and time_diff(ufcfightscrap['date'][i],day1)>0]
    for i in good_indices:
        summ+=ufcfightscrap[stat][i]
    t= time_in_octagon(guy,day1)
    if t==0:
        summ=0
    else:
        summ=summ/t
    return summ

#vectorize these functions
count_vect= npy.vectorize(count)
avg_count_vect= npy.vectorize(avg_count)

In [50]:
mask=ufcfighterscrap['dob']!='--'
for col in ufcfighterscrap.columns:
    maska=ufcfighterscrap[col]!='--'
    maskb=ufcfighterscrap[col]!='NaN'
    mask=mask&maska&maskb

In [51]:
datas=ufcfighterscrap[mask]

In [52]:
ufc_fighters=pd.DataFrame(data=datas,columns=ufcfighterscrap[mask].columns)

In [63]:
ufc_fighters

,dob,height,name,reach,stance,url,age
0,"Jan 19, 1990","5' 6""",Timur Valiev,"67""",Orthodox,http://ufcstats.com/fighter-details/45e738622e...,30
2,"Apr 26, 1993","5' 10""",Carlton Minus,"74""",Orthodox,http://ufcstats.com/fighter-details/40d8b74488...,27
3,"Oct 07, 1991","6' 2""",Jordan Wright,"77""",Orthodox,http://ufcstats.com/fighter-details/c878270af6...,28
4,"Aug 04, 1990","5' 7""",Trevin Jones,"70""",Southpaw,http://ufcstats.com/fighter-details/ec23c1be62...,30
5,"Aug 19, 1982","6' 4""",Stipe Miocic,"80""",Orthodox,http://ufcstats.com/fighter-details/d28dee5c70...,38
6,"Mar 20, 1979","5' 11""",Daniel Cormier,"72""",Orthodox,http://ufcstats.com/fighter-details/d967f0128c...,41
7,"Dec 02, 1992","5' 8""",Marlon Vera,"70""",Switch,http://ufcstats.com/fighter-details/7c7332319c...,27
8,"Oct 24, 1994","5' 11""",Sean O'Malley,"72""",Switch,http://ufcstats.com/fighter-details/b50a426a33...,25
9,"Mar 17, 1988","6' 4""",Jairzinho Rozenstruik,"78""",Orthodox,http://ufcstats.com/fighter-details/2cd428e960...,32
10,"Jan 30, 1984","6' 4""",Junior Dos Santos,"77""",Orthodox,http://ufcstats.com/fighter-details/63def5a566...,36


In [64]:
time wins_before('Chael Sonnen')

CPU times: user 250 ms, sys: 2.73 ms, total: 253 ms
Wall time: 259 ms


7

In [54]:
#making an age column for ufc_fighters
ufc_fighters['age']=age_vect(ufc_fighters['dob'])

In [ ]:
#the following should run in less than 4 minutes each

In [65]:
ufc_fighters['wins']=wins_before_vect(ufc_fighters['name'])

In [66]:
ufc_fighters['losses']=losses_before_vect(ufc_fighters['name'])

In [70]:
record_stats=[L5Y_wins_vect,L5Y_losses_vect, L2Y_wins_vect, L2Y_losses_vect,
                                                ko_wins_vect,ko_losses_vect,L5Y_ko_wins_vect,L5Y_ko_losses_vect,L2Y_ko_wins_vect,
                                                L2Y_ko_losses_vect,sub_wins_vect,sub_losses_vect,L5Y_sub_wins_vect,
                                                L5Y_sub_losses_vect, L2Y_sub_wins_vect, L2Y_sub_losses_vect]

In [71]:
for stat in record_stats:
    ufc_fighters[str(stat)]=stat(ufc_fighters['name'])
    print stat

In [72]:
punch_stats=[u'knockdowns',u'pass',u'reversals',u'sub_attempts',
                                                u'takedowns_landed',u'takedowns_attempts',u'sig_strikes_landed',
                                                u'sig_strikes_attempts',
                                                'total_strikes_landed',
                                                'total_strikes_attempts',
                                                'head_strikes_landed',
                                                'head_strikes_attempts',
                                                'body_strikes_landed',
                                                'body_strikes_attempts',
                                                'leg_strikes_landed',
                                                'leg_strikes_attempts']

In [73]:
for stat in punch_stats:
    col=stat+'_inf'
    ufc_fighters[col]=avg_count_vect(ufc_fighters['name'],stat,'inf')

KeyboardInterrupt: 

In [75]:
ufc_fighters

,dob,height,name,reach,stance,url,age,wins,losses,<numpy.vectorize object at 0x11a5c6d50>,...,sub_attempts_inf,takedowns_landed_inf,takedowns_attempts_inf,sig_strikes_landed_inf,sig_strikes_attempts_inf,total_strikes_landed_inf,total_strikes_attempts_inf,head_strikes_landed_inf,head_strikes_attempts_inf,body_strikes_landed_inf
0,"Jan 19, 1990","5' 6""",Timur Valiev,"67""",Orthodox,http://ufcstats.com/fighter-details/45e738622e...,30,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Apr 26, 1993","5' 10""",Carlton Minus,"74""",Orthodox,http://ufcstats.com/fighter-details/40d8b74488...,27,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Oct 07, 1991","6' 2""",Jordan Wright,"77""",Orthodox,http://ufcstats.com/fighter-details/c878270af6...,28,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Aug 04, 1990","5' 7""",Trevin Jones,"70""",Southpaw,http://ufcstats.com/fighter-details/ec23c1be62...,30,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Aug 19, 1982","6' 4""",Stipe Miocic,"80""",Orthodox,http://ufcstats.com/fighter-details/d28dee5c70...,38,14,3,7,...,0,0,0,0,0,0,0,0,0,0
6,"Mar 20, 1979","5' 11""",Daniel Cormier,"72""",Orthodox,http://ufcstats.com/fighter-details/d967f0128c...,41,11,3,6,...,0,0,0,0,0,0,0,0,0,0
7,"Dec 02, 1992","5' 8""",Marlon Vera,"70""",Switch,http://ufcstats.com/fighter-details/7c7332319c...,27,10,5,9,...,0,0,0,0,0,0,0,0,0,0
8,"Oct 24, 1994","5' 11""",Sean O'Malley,"72""",Switch,http://ufcstats.com/fighter-details/b50a426a33...,25,4,1,4,...,0,0,0,0,0,0,0,0,0,0
9,"Mar 17, 1988","6' 4""",Jairzinho Rozenstruik,"78""",Orthodox,http://ufcstats.com/fighter-details/2cd428e960...,32,5,1,5,...,0,0,0,0,0,0,0,0,0,0
10,"Jan 30, 1984","6' 4""",Junior Dos Santos,"77""",Orthodox,http://ufcstats.com/fighter-details/63def5a566...,36,15,7,4,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#EVERY COLUMN I"VE ADDED IS JUST ALL ZEROES!!!!! NEED TO FIGURE OUT WHAT WENT WRONG

In [ ]:
#I did the computation in my office and here is what we got:

In [76]:
ufc_fighters_office=pd.read_csv('ufc_fighters_office.csv',sep=',')

In [77]:
ufc_fighters_office

,dob,height,name,reach,stance,url,inf_knockdown_avg,pass_avg,reversals_avg,sub_attempts_avg,...,body_strikes_landed_avg,body_strikes_attempts_avg,leg_strikes_landed_avg,leg_strikes_attempts_avg,distance_strikes_landed_avg,distance_strikes_attempts_avg,clinch_strikes_landed_avg,clinch_strikes_attempts_avg,ground_strikes_landed_avg,ground_strikes_attempts_avg
0,"Aug 19, 1982","6' 4""",Stipe Miocic,"80""",Orthodox,http://ufcstats.com/fighter-details/d28dee5c70...,0.035833,0.102380,0.000000,0.000000,...,0.665472,0.762734,0.604044,0.644996,3.465575,7.248528,0.598925,0.890709,0.839519,1.115946
1,"Mar 20, 1979","5' 11""",Daniel Cormier,"72""",Orthodox,http://ufcstats.com/fighter-details/d967f0128c...,0.009642,0.144636,0.000000,0.038570,...,0.573724,0.805143,0.530333,0.665327,2.632382,5.664926,1.007634,1.369225,0.752109,0.988349
2,"Dec 02, 1992","5' 8""",Marlon Vera,"70""",Switch,http://ufcstats.com/fighter-details/7c7332319c...,0.031733,0.095198,0.000000,0.082505,...,0.932938,1.561244,0.913899,1.129681,2.589380,6.035541,0.539454,0.729850,0.571187,0.844087
3,"Oct 24, 1994","5' 11""",Sean O'Malley,"72""",Switch,http://ufcstats.com/fighter-details/b50a426a33...,0.025907,0.025907,0.025907,0.077720,...,1.295337,1.735751,0.466321,0.492228,5.699482,10.336788,0.181347,0.233161,0.466321,0.466321
4,"Mar 17, 1988","6' 4""",Jairzinho Rozenstruik,"78""",Orthodox,http://ufcstats.com/fighter-details/2cd428e960...,0.123203,0.000000,0.024641,0.000000,...,0.443532,0.566735,1.059548,1.232033,2.907598,6.431211,0.591376,0.788501,0.542094,0.665298
5,"Jan 30, 1984","6' 4""",Junior Dos Santos,"77""",Orthodox,http://ufcstats.com/fighter-details/63def5a566...,0.058467,0.020881,0.000000,0.004176,...,1.566089,2.309459,0.200459,0.242222,3.892253,8.339946,0.392566,0.676550,0.304865,0.496972
6,"Aug 06, 1985","5' 7""",Daniel Pineda,"69""",Orthodox,http://ufcstats.com/fighter-details/361d49960a...,0.016371,0.212824,0.065484,0.114598,...,0.507503,0.687585,0.343793,0.376535,1.620737,4.534789,0.229195,0.311050,1.604366,2.128240
7,"Feb 02, 1988","5' 9""",Herbert Burns,"73""",Orthodox,http://ufcstats.com/fighter-details/4f14040300...,0.073171,0.365854,0.000000,0.146341,...,0.585366,1.097561,0.146341,0.219512,0.951220,2.560976,0.292683,0.439024,0.365854,0.585366
8,"Jan 10, 1991","5' 6""",Merab Dvalishvili,"68""",Orthodox,http://ufcstats.com/fighter-details/c03520b5c8...,0.009524,0.228571,0.000000,0.019048,...,0.571429,0.885714,0.838095,0.923810,1.914286,6.704762,0.666667,0.933333,1.333333,2.238095
9,"Sep 26, 1984","5' 3""",John Dodson,"66""",Orthodox,http://ufcstats.com/fighter-details/99506df1af...,0.047499,0.008636,0.000000,0.000000,...,0.962936,1.265203,0.453401,0.505218,2.724721,7.185318,0.362720,0.531126,0.116589,0.151134


In [1]:
for i in range(1,20,2):
    print(i)

1
3
5
7
9
11
13
15
17
19


In [2]:
strin='123'

In [3]:
lst=[1,1,1,1,1]
lst[1]+=1
lst

[1, 2, 1, 1, 1]

In [4]:
lst[-1]

1

In [6]:
lst[-4]

2